In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
root = '../Yelp/'
filenames = [ele for ele in os.listdir(root) if ele.endswith('json')]
print(filenames)

['yelp_academic_dataset_checkin.json', 'yelp_academic_dataset_tip.json', 'yelp_academic_dataset_review.json', 'yelp_academic_dataset_business.json', 'yelp_academic_dataset_user.json']


# Prune `user`

In [4]:
# user
user_df = pd.read_json(os.path.join(root, "yelp_academic_dataset_user.json"), lines=True)
print(user_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB
None


* `fans` >= 10

In [27]:
print(f'Before: {user_df.shape}')
active_user_df = user_df.loc[user_df.fans >= 10]
print(f'After: {active_user_df.shape}')

Before: (1987897, 22)
After: (51588, 22)


* len(`elite`) > 0 

In [28]:
print(f'Before: {active_user_df.shape}')
active_user_df = active_user_df.loc[active_user_df.elite.str.len() > 0]
active_user_df = active_user_df.loc[active_user_df.elite.str.split(',').apply(lambda x: len(x)) > 0]
print(f'After: {active_user_df.shape}')

Before: (51588, 22)
After: (43494, 22)


In [29]:
active_user_df.to_json('rev_Yelp/yelp_academic_dataset_user.json')

# Prune `business`

In [32]:
# business
business_df = pd.read_json(os.path.join(root, "yelp_academic_dataset_business.json"), lines=True)
print(business_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB
None


* Delete null value

In [34]:
print(f'Before: {business_df.shape}')
business_df = business_df.loc[~business_df['hours'].isnull()]
business_df = business_df.loc[~business_df['categories'].isnull()]
business_df = business_df.loc[~business_df['attributes'].isnull()]
print(f'After: {business_df.shape}')

Before: (150346, 14)
After: (117618, 14)


* `categories` has "Restaurants"

In [35]:
print(f'Before: {business_df.shape}')
restaurant_df = business_df.loc[business_df['categories'].str.contains('Restaurants')]
print(f'After: {restaurant_df.shape}')

Before: (117618, 14)
After: (44676, 14)


* Delete `is_open` equal to 0

In [40]:
print(f'Before: {restaurant_df.shape}')
restaurant_df = restaurant_df.loc[restaurant_df['is_open'] == 1]
print(f'After: {restaurant_df.shape}')

Before: (44676, 14)
After: (31357, 14)


In [41]:
restaurant_df.to_json('rev_Yelp/yelp_academic_dataset_business.json')

# Load `user` and `business` json to prune other tables

In [4]:
# check
active_user_df = pd.read_json("rev_Yelp/yelp_academic_dataset_user.json")
print(active_user_df.info())
restaurant_df = pd.read_json("rev_Yelp/yelp_academic_dataset_business.json")
print(restaurant_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43494 entries, 0 to 1876575
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             43494 non-null  object 
 1   name                43494 non-null  object 
 2   review_count        43494 non-null  int64  
 3   yelping_since       43494 non-null  object 
 4   useful              43494 non-null  int64  
 5   funny               43494 non-null  int64  
 6   cool                43494 non-null  int64  
 7   elite               43494 non-null  object 
 8   friends             43494 non-null  object 
 9   fans                43494 non-null  int64  
 10  average_stars       43494 non-null  float64
 11  compliment_hot      43494 non-null  int64  
 12  compliment_more     43494 non-null  int64  
 13  compliment_profile  43494 non-null  int64  
 14  compliment_cute     43494 non-null  int64  
 15  compliment_list     43494 non-null  int64  
 16  co

## Prune `review`

In [5]:
# review
review_df = pd.read_json(os.path.join(root, "yelp_academic_dataset_review.json"), lines=True)
print(review_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB
None


* Delete non-existing `business_id` and `user_id`

In [6]:
print(f'Before: {review_df.shape}')
rev_review_df = review_df[review_df['user_id'].isin(active_user_df['user_id'].values)]
print(f'After: {rev_review_df.shape}')

Before: (6990280, 9)
After: (1067201, 9)


In [7]:
print(f'Before: {rev_review_df.shape}')
rev_review_df = rev_review_df[rev_review_df['business_id'].isin(restaurant_df['business_id'].values)]
print(f'After: {rev_review_df.shape}')

Before: (1067201, 9)
After: (550008, 9)


In [8]:
rev_review_df.to_json('rev_Yelp/yelp_academic_dataset_review.json')

# Prune `tip`

In [9]:
# tip
tip_df = pd.read_json(os.path.join(root, "yelp_academic_dataset_tip.json"), lines=True)
print(tip_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB
None


* Delete non-existing `business_id` and `user_id`

In [10]:
print(f'Before: {tip_df.shape}')
rev_tip_df = tip_df[tip_df['user_id'].isin(active_user_df['user_id'].values)]
print(f'After: {rev_tip_df.shape}')

Before: (908915, 5)
After: (197871, 5)


In [11]:
print(f'Before: {rev_tip_df.shape}')
rev_tip_df = rev_tip_df[rev_tip_df['business_id'].isin(restaurant_df['business_id'].values)]
print(f'After: {rev_tip_df.shape}')

Before: (197871, 5)
After: (87899, 5)


In [12]:
rev_tip_df.to_json('rev_Yelp/yelp_academic_dataset_tip.json')